In [29]:
!pip install h3pandas
!pip install folium
!pip install branca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.8 MB/s eta 0:00:00


In [34]:
import h3pandas
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm

In [5]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/311_Service_Requests_2007.csv")

In [6]:
data.head()

,Case Summary,Case Status,Case Source,Case Created Date,Case Created dttm,Case Closed Date,Case Closed dttm,First Call Resolution,Customer Zip Code,Incident Address 1,...,Latitude,Agency,Division,Major Area,Type,Topic,Council District,Police District,Neighborhood,ObjectId
0,Missed Trash Collection,Closed,Email,1/1/2007 12:00:00 AM,1/1/2007 11:42:01 AM,1/2/2007 12:00:00 AM,1/2/2007 8:34:20 AM,N,,3380 S Locust St,...,39.655934,Public Works,Solid Waste,Trash Collection,Information,Miss Trsh/Recy/Cmpst,4,3,Hampden,1
1,Snow Plowing or Ice Blading (warm transfer to ...,Closed,Web,1/1/2007 12:00:00 AM,1/1/2007 2:41:59 AM,1/2/2007 12:00:00 AM,1/2/2007 12:19:02 PM,N,,3832 W 1st Ave,...,39.717865,Public Works,Street Maint,Street/Alley,Service,Snow Control,3,4,Barnum,2
2,Loose Dog,Closed,Web,1/1/2007 12:00:00 AM,1/1/2007 1:44:08 PM,1/2/2007 12:00:00 AM,1/2/2007 8:00:17 AM,N,0,2228 S Fox St,...,39.676172,Env Health,Animl Care&Cntl,Animal Control,Complaint,Loose Dog,7,3,Overland,3
3,Recycling Information,Closed,Web,1/1/2007 12:00:00 AM,1/1/2007 6:17:42 AM,1/2/2007 12:00:00 AM,1/2/2007 8:05:58 AM,N,80246-1159,NaN,...,0.000000,Public Works,Solid Waste,Recycle-Collection,Information,Recycling Info,0,0,NaN,4
4,311 - General Inquiry,Closed,Email,1/1/2007 12:00:00 AM,1/1/2007 4:51:16 PM,1/2/2007 12:00:00 AM,1/2/2007 7:55:22 AM,N,,NaN,...,0.000000,Tech Services,311 Call Center,General Inquiry,Information,General Inquiry,0,0,NaN,5


In [16]:
data.shape

(379840, 25)

In [21]:
data["Case Summary"].value_counts().head(20)

,count
Case Summary,
311 - General Inquiry,30650
Out of Jurisdiction,19130
311 - Employee Transfer,14727
Police Department General Inquiry,9766
411 Information,8683
Large Item Pickup,8641
Pothole needs repair (queue case),8311
Excise and License License/permit requirements and types,8104
"Finding a lost pet, returning a found pet, or requesting a tag trace",7813


In [22]:
mask = data["Case Summary"] == "Loose Dog / Leash Law Violation"
prepared_data = data[mask]
prepared_data = prepared_data.dropna(subset=["Latitude", "Longitude"])
prepared_data.head()

,Case Summary,Case Status,Case Source,Case Created Date,Case Created dttm,Case Closed Date,Case Closed dttm,First Call Resolution,Customer Zip Code,Incident Address 1,...,Latitude,Agency,Division,Major Area,Type,Topic,Council District,Police District,Neighborhood,ObjectId
7605,Loose Dog / Leash Law Violation,Closed,Phone,1/8/2007 12:00:00 AM,1/8/2007 4:27:22 PM,1/8/2007 12:00:00 AM,1/8/2007 4:38:14 PM,N,,3300 S Tamarac Dr,...,39.656445,Env Health,Animl Care&Cntl,Animal Control,Complaint,Loose Dog,4,3,Hampden,7606
7621,Loose Dog / Leash Law Violation,Closed,Phone,1/8/2007 12:00:00 AM,1/8/2007 5:48:04 PM,1/8/2007 12:00:00 AM,1/8/2007 6:14:18 PM,N,,,...,39.760320,Env Health,Animl Care&Cntl,Animal Control,Complaint,Loose Dog,1,1,West Highland,7622
7624,Loose Dog / Leash Law Violation,Closed,Phone,1/8/2007 12:00:00 AM,1/8/2007 5:15:29 PM,1/8/2007 12:00:00 AM,1/8/2007 6:02:55 PM,N,80219-5918,2210 S Decatur St,...,39.676162,Env Health,Animl Care&Cntl,Animal Control,Complaint,Loose Dog,7,4,College View / South Platte,7625
7656,Loose Dog / Leash Law Violation,Closed,Phone,1/8/2007 12:00:00 AM,1/8/2007 3:35:56 PM,1/8/2007 12:00:00 AM,1/8/2007 3:40:36 PM,N,,,...,39.731215,Env Health,Animl Care&Cntl,Animal Control,Complaint,Loose Dog,3,1,Villa Park,7657
7659,Loose Dog / Leash Law Violation,Closed,Phone,1/8/2007 12:00:00 AM,1/8/2007 6:22:30 PM,1/8/2007 12:00:00 AM,1/8/2007 6:42:19 PM,N,,NaN,...,0.000000,Env Health,Animl Care&Cntl,Animal Control,Complaint,Loose Dog,0,0,NaN,7660


In [38]:
prepared_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6945 entries, 7605 to 379563
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Case Summary             6945 non-null   object 
 1   Case Status              6945 non-null   object 
 2   Case Source              6945 non-null   object 
 3   Case Created Date        6945 non-null   object 
 4   Case Created dttm        6945 non-null   object 
 5   Case Closed Date         6945 non-null   object 
 6   Case Closed dttm         6945 non-null   object 
 7   First Call Resolution    6945 non-null   object 
 8   Customer Zip Code        6945 non-null   object 
 9   Incident Address 1       5310 non-null   object 
 10  Incident Address 2       5310 non-null   object 
 11  Incident Intersection 1  5310 non-null   object 
 12  Incident Intersection 2  5310 non-null   object 
 13  Incident Zip Code        5310 non-null   object 
 14  Longitude               

In [32]:
prepared_data.shape

(6945, 25)

In [39]:
prepared_data["is_loose_Dog"] = 1

In [50]:
gdf = gpd.GeoDataFrame(
    prepared_data["is_loose_Dog"],
    geometry=gpd.points_from_xy(
        prepared_data.Latitude,
        prepared_data.Longitude
    )
)
gdf.head()

,is_loose_Dog,geometry
7605,1,POINT (39.656 -104.897)
7621,1,POINT (39.76 -105.043)
7624,1,POINT (39.676 -105.023)
7656,1,POINT (39.731 -105.044)
7659,1,POINT (0 0)


In [59]:
h3_data_9 = gdf.h3.geo_to_h3_aggregate(10, return_geometry=False).h3.h3_to_parent_aggregate(9)

In [60]:
h3_data_9.head()

,is_loose_Dog,geometry
h3_09,,
89754e64993ffff,1641,"POLYGON ((9.80613e-5 -0.00031, 0.00036 0.00133..."
89f22d9008fffff,1,"POLYGON ((-140.31012 -75.1655, -140.30638 -75...."
89f22d90107ffff,11,"POLYGON ((-140.21889 -75.15487, -140.21514 -75..."
89f22d9010bffff,2,"POLYGON ((-140.22825 -75.15967, -140.22449 -75..."
89f22d9010fffff,11,"POLYGON ((-140.21825 -75.15783, -140.2145 -75...."


In [58]:
m = folium.Map(
    location=[39.73, -105.044],
    zoom_start=11,
    tiles="cartodbpositron",
    attributionControl=False
)

folium.Choropleth(
    geo_data = h3_data_3,
    fill_opacite=0.3,
    line_weight=0.5
).add_to(m)

m
